In [33]:
import numpy as np
from azureml.core import Workspace, Dataset
from azureml.core import ScriptRunConfig, RunConfiguration, Experiment
from azureml.train.estimator import Estimator

In [15]:
ws = Workspace.from_config()

# Testing clean.py

In [51]:
dataset = Dataset.get_by_name(ws, name='annonces_ds')
experiment = Experiment(ws, "clean.py test")

est = Estimator(source_directory='.', entry_script='./pipeline_steps/clean.py',
                script_params= {"--input": 'annonces_ds', "--output": 'cleantest'},
                inputs=[dataset.as_named_input('annonces_ds')],
                compute_target='local',
                use_docker=False
               )

run = experiment.submit(est)

run.wait_for_completion(show_output=True)

WARNING - 'use_docker' parameter will be deprecated. Please use 'environment_definition' instead.


RunId: MyExperiment_1585683808_971875ca
Web View: https://ml.azure.com/experiments/MyExperiment/runs/MyExperiment_1585683808_971875ca?wsid=/subscriptions/68bdd703-8837-469c-80bd-bfb35f3b886f/resourcegroups/ProjectGroup2/workspaces/RealEstatePG2

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt

Streaming azureml-logs/70_driver_log.txt

Starting the daemon thread to refresh tokens in background for process with pid = 2307
Entering Run History Context Manager.
Preparing to call script [ pipeline_steps/clean.py ] with arguments: ['--input', 'annonces_ds', '--output', 'cleantest']
After variable expansion, calling script [ pipeline_steps/clean.py ] with arguments: ['--input', 'annonces_ds', '--output', 'cleantest']

/home/jovyan/.azureml/envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/python3.6/site-packages/azureml/dataprep/api/dataflow.py:722: UserWarning: Your pandas and pyarrow versions are incompatible. Please install pyarrow>=0.12.0 for 

{'runId': 'MyExperiment_1585683808_971875ca',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-03-31T19:43:31.601936Z',
 'endTimeUtc': '2020-03-31T19:45:01.404001Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'd5dcb806-b086-44ae-886d-012be2d0f4ca'},
 'inputDatasets': [{'dataset': {'id': '5a88c2bd-5165-485b-a81f-bc8290270b5a'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'annonces_ds', 'mechanism': 'Direct'}}],
 'runDefinition': {'script': 'pipeline_steps/clean.py',
  'useAbsolutePath': False,
  'arguments': ['--input', 'annonces_ds', '--output', 'cleantest'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'annonces_ds': {'dataLocation': {'dataset': {'id': '5a88c2bd-5165-485b-a81f-bc8290270b5a'},
     'dataPath': None},
    'createOutputDirectories': False,
    'mechanism': 'Direct',
    'environmentVariableName': 'annonces_ds',

In [56]:
run.download_file('outputs/cleantest', output_file_path='./cleantmp.csv')

# Testing .py